### BIM to GIS Conversion

#### This Notebook imports revit files from BIM models and convert them to GIS geodatabase in prepartion for monitoring
#### Please implement the following steps:
1. Define Projection to PRS92
2. Export Feature to file geodatabase
3. Merge all feature layers
4. Delete unwanted fields
5. Add Fields
6. Calculate Field for Type
7. Calculate Field for SubType
8. Delete Fields

In [1]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"
workspace = r'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexsc/During-Construction_nscrexsc.gdb'
#workspace = r'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'


In [2]:
# First, makre sure to Add to Current Map all the models:
## e.g. 'StructuralColumns', 'Roofs'

# Bulding name
building_name = "WRS"

fDataset = r'Depot_Building'

path = os.path.join(workspace, fDataset)

In [4]:
# 1. Define Projection to PRS92
## Add revit files to the Contents panel
## Note that 'Floors' in the BIM Model includes both slab and floors together.

columnA = r'Columns'
floor = r'Floors'
roof = r'Roofs'
framing = r'StructuralFraming'
foundation = r'StructuralFoundation'
column = r'StructuralColumns'

# Note that if we import from '-MOD-A-000001.rvt', it uses different 'BldgLevel' domain
# names and levels fro '-MOD-ST-000001.rvt'.
# It may be better for the GIS model to only utilize '-MOD-ST-000001.rvt' (i.e., drop 'roof')

if building_name in ("DHS", "LGS","TLOS","URS"):
    in_features = [roof, framing, foundation, column]
elif building_name in ("MPS","WRS"):
    in_features = [framing, foundation, column]
elif building_name in ("CMV", "DB1","MCS","TGB"):
    in_features = [floor, framing, foundation, column]
elif building_name in ("CPS", "CNT","DSP","LRS","OCC","SH1","TMO"):
    in_features = [roof, floor, framing, foundation, column]
elif building_name in ("BPS"):
    in_features = [roof, columnA, framing, foundation, column]
elif building_name in ("FP1"):
    in_features = [roof, floor, columnA, framing, foundation, column]
                      
sr = arcpy.SpatialReference("PRS 1992 Philippines Zone III")

for fl in in_features:
    # Define Projection first
    arcpy.DefineProjection_management(fl, sr)    


ExecuteError: ERROR 000289: Failed to alter spatial reference.
Failed to execute (DefineProjection).


In [53]:
# 2. Export to file geodatabase

arcpy.FeatureClassToGeodatabase_conversion(in_features, path)


<Result 'C:\\Users\\eiji.LAPTOP-KSD9P6CP\\OneDrive - Oriental Consultants Global JV\\Documents\\ArcGIS\\Projects\\During-Construction_nscrexsc\\During-Construction_nscrexsc.gdb\\Depot_Building'>

In [5]:
# 3. Merge all feature layers
arcpy.env.workspace = path
output_feature = "Depot_Building_merged" + "_" + building_name
arcpy.Merge_management(in_features, output_feature,"","")

<Result 'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexsc/During-Construction_nscrexsc.gdb\\Depot_Building\\Depot_Building_merged_WRS'>

In [6]:
# 4. Delete Fields
## 4.1 Obtain fields to be kept
fieldList = [f.name for f in arcpy.ListFields(output_feature)]

# find OBJECTID, as this is not deletable
reg = re.compile(r'OBJECTID*|Shape|Shape_Length|Shape_Area|Shape.STArea()|Shape.STLength()|GlobalID')
objField = list(filter(reg.match, fieldList))
keepFields = objField + ['Category', 'Family', 'FamilyType','ObjectID', 'ObjectId', 'Bldg_Name', 'BldgLevel', 'OmniClassDescription', 'BldgLevel_Desc', 'DocName', 'Discipline']

# Drop fields
dropFields = [e for e in fieldList if e not in keepFields]

## 4.2 Delete fields from feature layer
arcpy.DeleteField_management(output_feature, dropFields)

<Result 'Depot_Building_merged_WRS'>

In [7]:
# 5. Add Fields
addFields = ['Type', 'SubType', 'Status', 'StartDate', 'TargetDate', \
             'StructureType', 'StructureLevel', 'P6ID', 'Comment', \
            'GridX', 'GridY', 'CP', 'temp', 'Name']

for field in addFields:
    if field in ('StartDate', 'TargetDate'):
        arcpy.AddField_management(output_feature, field, "DATE", "", "", "", field, "NULLABLE", "")
    elif field in ('P6ID', 'Comment', 'GridX', 'GridY', 'CP', 'temp', 'Name'):
        arcpy.AddField_management(output_feature, field, "TEXT", "", "", "", field, "NULLABLE", "")
    else:
        arcpy.AddField_management(output_feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [12]:
# Each station may have different criteria
# 6. Add "temp" field 
## Some Subtypes (domain) are not automatically extractable from existing information.
## As such, we must extract matching letters for these SubTypes.


# Foundation
FGirder = ['FG\\d{2}','FG\\d{1}']
FBeam = ['B\\d{2}','B\\d{1}']
FSlab = ['Slab']
FoundationAll = FGirder + FBeam + FSlab

#
Brace = ['HB1']
Beam = ['B\\d{1}', 'b\\d{1}']
Girder = ['G\\d{1}']
Column = ['C\\d{1}']
Footing = ['P\\d{1}/F\\d{1}']
Turnbuckle = ['Turnbuckle']
LC = ['^LC']
OthersAll = Beam + Girder + Column + Footing + Turnbuckle + LC

joinText = FoundationAll + OthersAll

searchText = "|".join(joinText)

with arcpy.da.UpdateCursor(output_feature, ["FamilyType","temp"]) as cursor:
    for row in cursor:
        if row[0]:
            try:
                reg = re.search(searchText,row[0]).group()
            except AttributeError:
                reg = re.search(searchText,row[0])
            row[1] = reg
        else:
            row[1] = None
        cursor.updateRow(row)

In [3]:
# Delete rows if FamilyType includes 'TB1_|B1_|C1_|S1_|S2_|HSS|W8X|W8X'
with arcpy.da.UpdateCursor(output_feature, ["temp"]) as cursor:
    for row in cursor:
        if row[0] in Delete:
            cursor.deleteRow()

In [15]:
# Enter 'Type' and 'SubType'
# 'Type' field domain:
## 1. Foundations
## 2. Columns
## 3. Framing
## 4. Roofs
## 5. Floor

# 'SubType' field domain:
## 1. Pile
## 2. Pile Cap
## 3. Column
## 4. Foundation Girder []
## 5. Girder
## 6. Roof Girder
## 7. Foundation Beam
## 8. Beam
## 9. Brace
## 10. Fire Exit Stair
## 11. Steel Staircase
## 12. Slab
## 13. Roof Beam

#
FGirder = ['FG1']
Beam = ['B1','B2']
Girder = ['FG1']
Turnbuckle = ['Turnbuckle']
Footing = ['P1/F1']
LC = ['LC']

with arcpy.da.UpdateCursor(output_feature, ['Category', 'Type', 'SubType','FamilyType', 'OmniClassDescription','temp','Family']) as cursor:
    for row in cursor:
        if row[0] == 'Structural Foundations' and row[4] == 'Foundation Piles': # Pile and Pile Cap as one
            row[1] = 1
            row[2] = 1
        elif row[4] == 'Columns': # Columns
            row[1] = 2
            row[2] = 3
        elif row[5] in FGirder: # Foundation Girder
            row[1] = 3
            row[2] = 4
        elif row[5] in GirderAll: # Girder
            row[1] = 3
            row[2] = 5
        elif row[5] in RoofGirder: # Roof Girder
            row[1] = 3
            row[2] = 6
        elif row[5] in FBeam: # Foundation Beam
            row[1] = 3
            row[2] = 7
        elif row[5] in BeamAll: # Beam
            row[1] = 3
            row[2] = 8
        elif row[5] in Brace: # Brace
            row[1] = 3
            row[2] = 9
        elif row[5] in FireExitStair: # Fire Exit Stair
            row[1] = 3
            row[2] = 10
        elif row[5] in SteelStaircase: # Steel Staircase
            row[1] = 3
            row[2] = 11
        elif row[6] == 'Floor':
            row[1] = 5
            row[2] = 12
        elif row[5] in RoofBeam: # Roof Beam
            row[1] = 3
            row[2] = 13
        else:
            row[1] = None
            row[2] = None
        cursor.updateRow(row)

In [13]:
# Enter 'StructureType' and 'StructureLevel' from P6
# StructureType
## 1. Substructure
## 2. Superstructure

# StructureLevel
## 1. Foundation
## 2. Ground Level
## 3. Concourse Level
## 4. Platform Level
## 5. Roof Level

# 'SubType' field domain:
## 1. Pile
## 2. Pile Cap
## 3. Column
## 4. Foundation Girder []
## 5. Girder
## 6. Roof Girder
## 7. Foundation Beam
## 8. Beam
## 9. Brace
## 10. Fire Exit Stair
## 11. Steel Staircase
## 12. Slab
## 13. Roof Beam


# First fill 'StructureLevel'
with arcpy.da.UpdateCursor(output_feature, ['SubType', 'temp', 'StructureLevel']) as cursor:
    for row in cursor:
        # Concorse Level
        if row[1] in ConcourseLevel:
            row[2] = 3
        # Platform Level
        elif row[1] in PlatformLevel:
            row[2] = 4
        # Roof Level
        elif row[1] in RoofLevel:
            row[2] = 5
                # 'Foundation' and 'Substructure'
        elif row[1] in Foundation_Substr: 
            row[2] = 1
        else:
            row[2] = None
        cursor.updateRow(row)

# *********************************************************
# manually update StructureLevel for Foundation Beam.
## B150 familytype occurs in both Foundation and Platform level
# *********************************************************

# Then 'StructureType'
with arcpy.da.UpdateCursor(output_feature, ['StructureType', 'StructureLevel']) as cursor:
    for row in cursor:
        if row[1] == 1:
            row[0] = 1
        else:
            row[0] = 2
        cursor.updateRow(row)

In [2]:
# Add some comments if necessary to 'Comment' field
output_feature = "Station_Structure_merged_Clark"
with arcpy.da.UpdateCursor(output_feature, ['temp','Comment']) as cursor:
    for row in cursor:
        if row[0] in ('B4','B5','G4','G5', 'C1'):# for Clark station
            row[1] = "Roof Framing Plan"
        else:
            row[1] == row[1]
        cursor.updateRow(row)
        

In [14]:
# Delete observations with SubType = None
with arcpy.da.UpdateCursor(output_feature, ['SubType']) as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

In [3]:
# Set the initial status = 'To be Constructed'
with arcpy.da.UpdateCursor(output_feature, ['Status']) as cursor:
    for row in cursor:
        row[0] = 1
        cursor.updateRow(row)

In [6]:
# Enter station name
with arcpy.da.UpdateCursor(output_feature, ['Name']) as cursor:
    for row in cursor:
        if Name == "BPS":
            row[0] = 1
        elif Name == "Depot":
            row[0] = 2
        elif Name == "CIA":
            row[0] = 3
        elif Name == "Clark":
            row[0] = 4
        elif Name == "Angeles":
            row[0] = 5
        elif Name == "SanFernando":
            row[0] = 6
        elif Name == "Apalit":
            row[0] = 7
        elif Name == "Calumpit":
            row[0] = 8
        elif Name == "Malolos":
            row[0] = 9
        elif Name == "Solis":
            row[0] = 10
        elif Name == "Blumentritt":
            row[0] = 11
        elif Name == "Espana":
            row[0] = 12
        elif Name == "Santa Mesa":
            row[0] = 13
        elif Name == "Paco":
            row[0] = 14
        elif Name == "Buendia":
            row[0] = 15
        elif Name == "EDSA":
            row[0] = 16
        elif Name == "Nichols":
            row[0] = 17
        elif Name == "FTI":
            row[0] = 18
        elif Name == "Bicutan":
            row[0] = 19
        elif Name == "Sucat":
            row[0] = 20
        elif Name == "Alabang":
            row[0] = 21
        elif Name == "Muntinlupa":
            row[0] = 22
        elif Name == "San Pedro":
            row[0] = 23
        elif Name == "Pacita":
            row[0] = 24
        elif Name == "Binan":
            row[0] = 25
        elif Name == "Santa Rosa":
            row[0] = 26
        elif Name == "Cabuyao":
            row[0] = 27
        elif Name == "Banlic Depot":
            row[0] = 28
        elif Name == "Banlic":
            row[0] = 29
        elif Name == "Calamba":
            row[0] = 30
        cursor.updateRow(row)


In [7]:
# Assign Domains
# 0. Domain Name
domainList = ['Station Structures_TYPE', 'Station Structures_SUBTYPE',\
              'Station Structures_STATUS', 'Station Structures_STRUCTURETYPE',\
              'Station Structures_STRUCTURELEVEL','Station_nscrex']

#domains = arcpy.da.ListDomains(workspace)
#listDomain = [d.name for d in domains]
#reg = re.compile(r'Station Structures*|Station*')
#listFinal = list(filter(reg.match, listDomain))

arcpy.AssignDomainToField_management(output_feature, 'Type', domainList[0])
arcpy.AssignDomainToField_management(output_feature, 'SubType', domainList[1])
arcpy.AssignDomainToField_management(output_feature, 'Status', domainList[2])
arcpy.AssignDomainToField_management(output_feature, 'StructureType', domainList[3])
arcpy.AssignDomainToField_management(output_feature, 'StructureLevel', domainList[4])
arcpy.AssignDomainToField_management(output_feature, 'Station', domainList[5])

<Result 'Station_Structure_merged_CIA'>

In [18]:
# Apply Symbology
symbolLyrx = r'C:/Users/emasu/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structures.lyrx'
arcpy.ApplySymbologyFromLayer_management(output_feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [19]:
# Alter field. Rename a field name 'ObjectId' to 'ObjectId1'. The GIS table has OBJECTID, which is reserved name.
# We cannot use this name.
arcpy.AlterField_management(output_feature, 'ObjectId', 'ObjectId1', 'ObjectId1', 'TEXT')


<Result 'Station_Structure_merged_Apalit'>

In [20]:
# Delete Fields
dropF = ['Bldg_Name', 'OmniClassDescription', 'temp']
arcpy.DeleteField_management(output_feature, dropF)

<Result 'Station_Structure_merged_Apalit'>

In [21]:
# Add 'Grid' and 'Zone' for monitoring. this is used in reference to P6, too
addFields = ['Grid', 'Zone', 'ID']

for field in addFields:
    arcpy.AddField_management(output_feature, field, "TEXT", "", "", "", field, "NULLABLE","")